## Part 1: Preprocessing

In [1]:
# Install required packages
%pip install pandas numpy scikit-learn tensorflow

# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras import layers

#  Import and read the attrition data
attrition_df = pd.read_csv('https://static.bc-edx.com/ai/ail-v-1-0/m19/lms/datasets/attrition.csv')
attrition_df.head()

Note: you may need to restart the kernel to use updated packages.


,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EnvironmentSatisfaction,HourlyRate,JobInvolvement,...,PerformanceRating,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,Sales,1,2,Life Sciences,2,94,3,...,3,1,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,Research & Development,8,1,Life Sciences,3,61,2,...,4,4,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,Research & Development,2,2,Other,4,92,2,...,3,2,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,Research & Development,3,4,Life Sciences,4,56,3,...,3,3,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,Research & Development,2,1,Medical,1,40,3,...,3,4,1,6,3,3,2,2,2,2


In [2]:
# Determine the number of unique values in each column
attrition_df.nunique()

Age                         43
Attrition                    2
BusinessTravel               3
Department                   3
DistanceFromHome            29
Education                    5
EducationField               6
EnvironmentSatisfaction      4
HourlyRate                  71
JobInvolvement               4
JobLevel                     5
JobRole                      9
JobSatisfaction              4
MaritalStatus                3
NumCompaniesWorked          10
OverTime                     2
PercentSalaryHike           15
PerformanceRating            2
RelationshipSatisfaction     4
StockOptionLevel             4
TotalWorkingYears           40
TrainingTimesLastYear        7
WorkLifeBalance              4
YearsAtCompany              37
YearsInCurrentRole          19
YearsSinceLastPromotion     16
YearsWithCurrManager        18
dtype: int64

In [3]:
# Create y_df with the Attrition and Department columns
y_df = attrition_df[['Attrition', 'Department']]
y_df.head()


,Attrition,Department
0,Yes,Sales
1,No,Research & Development
2,Yes,Research & Development
3,No,Research & Development
4,No,Research & Development


In [4]:
# Create a list of at least 10 column names to use as X data
feature_columns = [
    'Age', 'DistanceFromHome', 'Education', 'EnvironmentSatisfaction',
    'HourlyRate', 'JobInvolvement', 'JobLevel', 'JobSatisfaction',
    'NumCompaniesWorked', 'PercentSalaryHike', 'PerformanceRating',
    'RelationshipSatisfaction', 'TotalWorkingYears', 'WorkLifeBalance',
    'YearsAtCompany'
]

# Create X_df using your selected columns
X_df = attrition_df[feature_columns]

# Show the data types for X_df
print(X_df.dtypes)


Age                         int64
DistanceFromHome            int64
Education                   int64
EnvironmentSatisfaction     int64
HourlyRate                  int64
JobInvolvement              int64
JobLevel                    int64
JobSatisfaction             int64
NumCompaniesWorked          int64
PercentSalaryHike           int64
PerformanceRating           int64
RelationshipSatisfaction    int64
TotalWorkingYears           int64
WorkLifeBalance             int64
YearsAtCompany              int64
dtype: object


In [5]:
# Split the data into training and testing sets
from sklearn.model_selection import train_test_split

# Split the X and y data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_df, y_df, random_state=42)


In [6]:
# Convert your X data to numeric data types however you see fit
# Add new code cells as necessary

# All features are already in numeric format (int64), which can be verified by:
print("All features are already in numeric format:")
print(X_df.dtypes.value_counts())

All features are already in numeric format:
int64    15
Name: count, dtype: int64


In [7]:
# Create a StandardScaler
scaler = StandardScaler()

# Fit the StandardScaler to the training data
scaler.fit(X_train)

# Scale the training and testing data
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [8]:
from sklearn.preprocessing import OneHotEncoder

# Create a OneHotEncoder for the Department column
department_encoder = OneHotEncoder(sparse_output=False)

# Fit the encoder to the training data
department_encoder.fit(y_train[['Department']])

# Create two new variables by applying the encoder
# to the training and testing data
department_train_encoded = department_encoder.transform(y_train[['Department']])
department_test_encoded = department_encoder.transform(y_test[['Department']])
print(department_train_encoded.shape)


(1102, 3)


In [9]:
# Create a OneHotEncoder for the Attrition column
attrition_encoder = OneHotEncoder(sparse_output=False)

# Fit the encoder to the training data
attrition_encoder.fit(y_train[['Attrition']])

# Create two new variables by applying the encoder
# to the training and testing data
attrition_train_encoded = attrition_encoder.transform(y_train[['Attrition']])
attrition_test_encoded = attrition_encoder.transform(y_test[['Attrition']])
print(attrition_train_encoded.shape)

(1102, 2)


## Part 2: Create, Compile, and Train the Model

In [17]:
# Find the number of columns in the X training data.
number_input_features = len(X_train.columns)

# Create the input layer
inputs = layers.Input(shape=(number_input_features,))

# Create at least two shared layers
hidden_layer1 = layers.Dense(units=128, activation='relu')(inputs)
hidden_layer2 = layers.Dense(units=64, activation='relu')(hidden_layer1)
shared_layer = layers.Dense(units=32, activation='relu')(hidden_layer2)


In [18]:
# Create a branch for Department
# with a hidden layer and an output layer

# Create the hidden layer
department_layer = layers.Dense(units=16, activation='relu')(shared_layer)

# Create the output layer (3 units for 3 departments, softmax for multi-class classification)
department_output = layers.Dense(units=3, activation='softmax', name='department_output')(department_layer)


In [19]:
# Create a branch for Attrition
# with a hidden layer and an output layer

# Create the hidden layer
attrition_layer = layers.Dense(units=16, activation='relu')(shared_layer)

# Create the output layer (2 units for binary classification with one-hot encoding, softmax activation)
attrition_output = layers.Dense(units=2, activation='softmax', name='attrition_output')(attrition_layer)


In [20]:
# Create the model
model = Model(inputs=inputs, outputs=[department_output, attrition_output])

# Compile the model
model.compile(
    optimizer='adam',
    loss={
        'department_output': 'categorical_crossentropy',
        'attrition_output': 'categorical_crossentropy'
    },
    metrics={
        'department_output': 'accuracy',
        'attrition_output': 'accuracy'
    }
)

# Summarize the model
model.summary()

# Train the model
history = model.fit(
    X_train_scaled,
    {
        'department_output': department_train_encoded,
        'attrition_output': attrition_train_encoded
    },
    epochs=50,
    batch_size=32,
    validation_split=0.2,
    verbose=1
)

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1       │ (None, 15)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_5 (Dense)     │ (None, 128)       │      2,048 │ input_layer_1[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_6 (Dense)     │ (None, 64)        │      8,256 │ dense_5[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_7 (Dense)     │ (None, 32)        │      2,080 │ dense_6[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_8 (Dense)     │ (None, 16)        │        528 │ dense_7[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_9 (Dense)     │ (None, 16)        │        528 │ dense_7[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ department_output   │ (None, 3)         │         51 │ dense_8[0][0]     │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ attrition_output    │ (None, 2)         │         34 │ dense_9[0][0]     │
│ (Dense)             │                   │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 13,525 (52.83 KB)

 Trainable params: 13,525 (52.83 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/50
28/28 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - attrition_output_accuracy: 0.6813 - attrition_output_loss: 0.5991 - department_output_accuracy: 0.6049 - department_output_loss: 0.9182 - loss: 1.5179 - val_attrition_output_accuracy: 0.7873 - val_attrition_output_loss: 0.5040 - val_department_output_accuracy: 0.6063 - val_department_output_loss: 0.8451 - val_loss: 1.3516
Epoch 2/50
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - attrition_output_accuracy: 0.8494 - attrition_output_loss: 0.4188 - department_output_accuracy: 0.6971 - department_output_loss: 0.7293 - loss: 1.1479 - val_attrition_output_accuracy: 0.7873 - val_attrition_output_loss: 0.4883 - val_department_output_accuracy: 0.6063 - val_department_output_loss: 0.8266 - val_loss: 1.3172
Epoch 3/50
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - attrition_output_accuracy: 0.8304 - attrition_output_loss: 0.4192 - department_output_accuracy: 0.6808 - department_output_loss: 0.7129 - loss: 1.1326 - val_attrition_output_accuracy: 0.7873 - va

In [22]:
from tabulate import tabulate

# Create a table with training and validation metrics
epochs = len(history.history['loss'])
final_epoch = epochs - 1

# Prepare data for the table
metrics = [
    ['Total Loss', history.history['loss'][final_epoch], history.history['val_loss'][final_epoch]],
    ['Department Accuracy', history.history['department_output_accuracy'][final_epoch], 
     history.history['val_department_output_accuracy'][final_epoch]],
    ['Attrition Accuracy', history.history['attrition_output_accuracy'][final_epoch], 
     history.history['val_attrition_output_accuracy'][final_epoch]]
]

# Display the table
print("\nFinal Model Metrics:")
print(tabulate(metrics, 
              headers=['Metric', 'Training', 'Validation'],
              tablefmt='grid',
              floatfmt='.4f'))



Final Model Metrics:
+---------------------+------------+--------------+
| Metric              |   Training |   Validation |
+=====================+============+==============+
| Total Loss          |     0.0108 |       4.7126 |
+---------------------+------------+--------------+
| Department Accuracy |     1.0000 |       0.5656 |
+---------------------+------------+--------------+
| Attrition Accuracy  |     1.0000 |       0.7738 |
+---------------------+------------+--------------+


In [23]:
# Evaluate the model with the testing data
test_loss = model.evaluate(
    X_test_scaled,
    {
        'department_output': department_test_encoded,
        'attrition_output': attrition_test_encoded
    },
    verbose=0
)

# Create a dictionary of metric names
metric_names = ['Total Loss', 'Department Loss', 'Attrition Loss', 'Department Accuracy', 'Attrition Accuracy']

# Print all metrics
for name, value in zip(metric_names, test_loss):
    print(f'{name}: {value:.4f}')

Total Loss: 4.1778
Department Loss: 2.7203
Attrition Loss: 1.5036
Department Accuracy: 0.7908
Attrition Accuracy: 0.5870


In [24]:
from tabulate import tabulate

# Create a table of the accuracies

# Create data for the table
data = [
    ['Department', f'{test_loss[3]:.1%}'],
    ['Attrition', f'{test_loss[4]:.1%}']
]

# Display the table
print("\nModel Accuracy by Output:")
print(tabulate(data, headers=['Output', 'Accuracy'], tablefmt='grid'))



Model Accuracy by Output:
+------------+------------+
| Output     | Accuracy   |
+============+============+
| Department | 79.1%      |
+------------+------------+
| Attrition  | 58.7%      |
+------------+------------+


# Summary

In the provided space below, briefly answer the following questions.

1. Is accuracy the best metric to use on this data? Why or why not?

2. What activation functions did you choose for your output layers, and why?

3. Can you name a few ways that this model might be improved?

YOUR ANSWERS HERE

1. Accuracy alone may not be reliable if the dataset has class imbalance. It can give a false sense of performance by favoring the majority class. Using precision, recall, or F1-score offers a more complete view of model effectiveness.
2. Softmax was used for both output layers to handle classification. It converts outputs into probabilities across classes, which works well with categorical_crossentropy. This makes it ideal for both multi-class (department) and binary (attrition) targets.
3. Apply class balancing techniques to address potential skewed distributions. Add dropout or L2 regularization to combat overfitting seen in validation metrics. Experiment with more features, deeper networks, or hyperparameter tuning to boost generalization.